Basics
------
This covers a fundamental use-case for DN<sup>3</sup>. Here, the TIDNet architecture from *Kostas & Rudzicz 2020
(https://doi.org/10.1088/1741-2552/abb7a7)* is evaluated using the Physionet motor dataset in a
*"leave multiple subjects out"* (aka person-stratified cross validation) training procedure.
This means, the number of subjects are divided into approximately evenly numbered sets, and the test performance of
each set is evaluated. The remaining sets for each test will be used to develop a neural network classifier.

Here we will focus on classifying imagined hand vs. foot movement, which are the runs 6, 10 and 14. So create a
configuration for the configuratron that specifies a single dataset, the length of the trials to cut for the dataset
 (6s), the events we are looking for (T1 and T2), and then to exclude all those sessions that are not 6, 10 and 14
(the sessions by default are listed as "S{subject number}R{run number}.edf". Finally, due to some issues with a few
people's recordings in the dataset, we can ignore those troublemakers. The following is the contents of a file named
"my_config.yml".

```yaml
Configuratron:
  preload: True

use_gpu: False

mmidb:
  name: "Physionet MMIDB"
  toplevel: /path/to/eegmmidb
  tmin: 0
  tlen: 6
  data_max: 0.001
  data_min: -0.001
  events:
    - T1
    - T2
  exclude_sessions:
    - "*R0[!48].edf"  # equivalently "*R0[1235679].edf"
    - "*R1[!2].edf"   # equivalently "*R1[134].edf"
  exclude_people:
    - S088
    - S090
    - S092
    - S100
  train_params:
    epochs: 7
    batch_size: 4
  lr: 0.0001
  folds: 5
```

Notice that in addition to the dataset and special `Configuratron` section we also include a `train_params` with the
dataset.
This last part is definitely *not mandatory*, it is an arbitrarily named additional set of configuration values that
will be used in our experiment, put there for convenience. See it in action below, but if you don't like it,
don't use it.

In [1]:
from dn3.configuratron import ExperimentConfig
from dn3.trainable.processes import StandardClassification
from dn3.trainable.models import TIDNet

# Since we are doing a lot of loading, this is nice to suppress some tedious information
import mne
mne.set_log_level(False)

First things first, we use `ExperimentConfig`, and the subsequently constructed `DatasetConfig` to rapidly construct
our `dataset`.

In [2]:
config_filename = 'my_config_copy.yml'
experiment = ExperimentConfig(config_filename)
ds_config = experiment.datasets['bci_iv_2a']

Adding additional configuration entries: dict_keys(['extensions', 'test_subjects', 'train_params', 'lr'])
Configuratron found 1 datasets.


In [ ]:
dataset = ds_config.auto_construct_dataset()

Let's create a function that makes a new model for each set of training people and a trainable process for
`StandardClassification`. The `cuda` argument handles whether the GPU is used to train the neural network if a
cuda-compatible PyTorch installation is operational.

In [ ]:
def make_model_and_process():
    tidnet = TIDNet.from_dataset(dataset)
    return StandardClassification(tidnet, cuda=experiment.use_gpu, learning_rate=ds_config.lr)

That's pretty much it! We use a helper that initializes a TIDNet from any `Dataset/Thinker/EpochedRecording` and wrap
it with a `StandardClassification` process. Invoking this process will train the classifier.
Have a look through all the `trainable.processes`, they can wrap the *same model* to learn in stages (e.g. some sort
 of fine-tuning procedure from a general model -- covered in `examples/finetuning.ipynb`).

Now, we loop through five folds, *leaving multiple subjects out* (`dataset.lmso()`), fit the classifier,
then check the test results.

In [ ]:
vars(ds_config.train_params)

In [ ]:
results = []

for training, validation, test in dataset.lmso(ds_config.folds):
    process = make_model_and_process()

    process.fit(training_dataset=training, validation_dataset=validation, **ds_config.train_params)

    results.append(process.evaluate(test)['Accuracy'])

print(results)
print("Average accuracy: {:.2%}".format(sum(results)/len(results)))

Check out how we passed the train_params to `.fit()`, we could specify more arguments for `.fit()` by just adding them
to this section in the configuration.

Say we wanted to *instead*, get the performance of *each* person in the test fold independently. We could replace the
code above with a very simple alternative, that *leaves one subject out* or `.loso()`, that specifically.
It would look like:

In [ ]:
results = dict()
for training, validation, test in dataset.lmso(ds_config.folds):
    process = make_model_and_process()

    process.fit(training_dataset=training, validation_dataset=validation,
                epochs=ds_config.train_params.epochs,
                batch_size=ds_config.train_params.batch_size)

    for _, _, test_thinker in test.loso():
        results[test_thinker.person_id] = process.evaluate(test_thinker)

print(results)

In [ ]:
avg_acc = sum(v['Accuracy'] for v in results.values()) / len(results)
print("Average accuracy: {:.2%}".format(avg_acc))

Try filling in your own values to `my_config.yml` to run these examples on your next read through.